In [1]:
import sys
import os 
sys.path.append('/home/potzschf/repos/')
from helperToolz.helpsters import *
from helperToolz.dicts_and_lists import *

In [8]:
# check if downloaded ERA5 variables acutally contain more than just one value for the study period
base_path = '/data/Aldhani/eoagritwin/et/Auxiliary/ERA5/grib/'
ssrd_files = [file for file in getFilelist(base_path, '.grib', deep=True) if 'downward' in file]
outPath = '/data/Aldhani/eoagritwin/et/Auxiliary/ERA5/ssrd_mean_calc/'

In [9]:
for ssrd_file in ssrd_files:
    stack, bands = stackReader(ssrd_file, True, True)

    if '2019_7.' in ssrd_file:
        ref_ds = checkPath(ssrd_file)
        ref_band = ref_ds.GetRasterBand(1)
    
        out_ds = gdal.GetDriverByName('GTiff').Create(f'{outPath}ssrd_2019_7_1.tif', ref_ds.RasterXSize, ref_ds.RasterYSize, 24, ref_band.DataType)
        out_ds.SetGeoTransform(ref_ds.GetGeoTransform())
        out_ds.SetProjection(ref_ds.GetProjection())
        for i in range(24):
            out_ds.GetRasterBand(i+1).WriteArray(stack[:,:,i])
            out_ds.GetRasterBand(i+1).SetDescription(bands[i])
        del out_ds

    # sanity check
    for i in range(0, stack.shape[2], 24):
        if not bands[i].split(' ')[-1] == '00:00:00':
            raise ValueError('Something is messy about this file')
    start = [i for i in range(0, stack.shape[2], 24)]
    end = start[1:] + [stack.shape[2]]
    if not len(start) == len(end) == int(stack.shape[2]/24):
        raise ValueError('Something wrong with the dates and times')
    
    contiL = []
    names = []
    for count, i in enumerate(start):
        contiL.append(np.nanmean(stack[:,:,i:end[count]], axis=2))
        names.append(f'{bands[i].split(' ')[0]}_mean')
    arr = np.dstack(contiL)

   
    ref_ds = checkPath(ssrd_file)
    ref_band = ref_ds.GetRasterBand(1)
    
    out_ds = gdal.GetDriverByName('GTiff').Create(f'{outPath}{ssrd_file.split('.')[0].split('/')[-1]}', ref_ds.RasterXSize, ref_ds.RasterYSize, len(names), ref_band.DataType)
    out_ds.SetGeoTransform(ref_ds.GetGeoTransform())
    out_ds.SetProjection(ref_ds.GetProjection())
    for i in range(len(names)):
        out_ds.GetRasterBand(i+1).WriteArray(contiL[i])
        out_ds.GetRasterBand(i+1).SetDescription(names[i])
    del out_ds

/data/Aldhani/users/potzschf/conda/envs/cds_era5/lib/python3.12/site-packages/osgeo/gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(
